Referensi

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression
https://ml2021.medium.com/chi-square-and-anova-feature-selection-for-ml-5e1063ab0991

In [1]:
# Import Library
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import time
from sklearn.feature_selection import f_regression,SelectKBest 
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
#import dataset
df = pd.read_csv('/home/jupyter-17523142/Rama/Dataset/dataset_rama.csv')
X = df.drop(['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis = 1)
y_vitc = df.loc[:, 'Vit C (mg/100g)']

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y_vitc, test_size = 0.3, random_state=0)

# Seleksi Fitur ANOVA

In [3]:
# Define ANOVA feature selection
anova = SelectKBest(score_func=f_regression, k=100)

In [4]:
%%time

# Apply feature selection
anova.fit(x_train,y_train)

CPU times: user 881 ms, sys: 46.6 ms, total: 928 ms
Wall time: 94.8 ms


SelectKBest(k=100, score_func=<function f_regression at 0x7ff11c15c268>)

In [5]:
dir(anova)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_params',
 '_get_param_names',
 '_get_support_mask',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_transform',
 '_validate_data',
 'feature_names_in_',
 'fit',
 'fit_transform',
 'get_feature_names_out',
 'get_params',
 'get_support',
 'inverse_transform',
 'k',
 'n_features_in_',
 'pvalues_',
 'score_func',
 'scores_',
 'set_params',
 'transform']

In [6]:
# Create features score, features p_value, and features name
features_score = pd.DataFrame(anova.scores_)
features_pvalue = pd.DataFrame(np.round(anova.pvalues_,4))
features = pd.DataFrame(x_train.columns)
features_anova = pd.concat([features,features_score,features_pvalue],axis=1)

# Assign the column name
features_anova.columns = ["Features","Score","P_Value"]

# Print features score
print(features_anova.nlargest(100,columns="Score"))

    Features     Score  P_Value
117   1047.1  2.452869   0.1198
116   1046.7  2.426808   0.1217
815   1458.2  2.423241   0.1220
819   1461.5  2.423166   0.1220
806   1450.9  2.421868   0.1221
..       ...       ...      ...
841   1479.9  2.323819   0.1299
72    1028.5  2.323106   0.1299
842   1480.7  2.322895   0.1299
66      1026  2.322411   0.1300
779   1429.3  2.321689   0.1300

[150 rows x 3 columns]


In [7]:
# Fitur yang diseleksi
print(features_anova.nlargest(100,columns="Score").iloc[:,0].values.flatten().tolist())

['1047.1', '1046.7', '1458.2', '1461.5', '1450.9', '1451.7', '1459.1', '1452.5', '1446.8', '1446', '1045', '1453.3', '1459.9', '1460.7', '1454.1', '1051.8', '1462.3', '1450.1', '1457.4', '1445.2', '1051', '1037.9', '1447.6', '1455', '1456.6', '1455.8', '1442.8', '1463.2', '1449.3', '1036.7', '1448.5', '1441.2', '1442', '1443.6', '1444.4', '1464', '1050.5', '1047.6', '1060', '1464.8', '1052.2', '1048.8', '1470.6', '1054.4', '1049.3', '1038.3', '1466.5', '1469', '1440.4', '1465.6', '1471.5', '1045.5', '1468.1', '1469.8', '1467.3', '1437.2', '1040.4', '1042.5', '1439.6', '1438', '1059.6', '1050.1', '1033.8', '1051.4', '1046.3', '1057.8', '1032.6', '1438.8', '1472.3', '1049.7', '1054', '1033.4', '1473.1', '1058.3', '1436.4', '1056.1', '1056.5', '1038.8', '1040', '1474', '1034.6', '1036.3', '1031.7', '1039.6', '1042.9', '1048.4', '1433.2', '1434', '1435.6', '1037.5', '1434.8', '1043.8', '1474.8', '1044.6', '1037.1', '1060.4', '1044.2', '1048', '1476.5', '1040.8']


In [8]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# Linear Regression

In [9]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    print("==================================================")
    start_time = time.time()

    #ambil n fitur input hasil seleksi fitur Anova
    x_train_selected = x_train[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]
    x_test_selected = x_test[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]

    #Create a Linear Regression
    lr_model = LinearRegression()

    #Train the model using the training sets
    lr_model.fit(x_train_selected, y_train)
    y_pred_lr = lr_model.predict(x_test_selected)

    #hitung score model dari data train
    scores = cross_validate(lr_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

    print("MSE model Linear Regression data Train dengan " + str(nfeat) + " fitur: " 
          + str(abs(round(scores['train_mse'].mean(), 2))))
    print("RMSE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_rmse'].mean(), 2))))
    print("MAE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_mae'].mean(), 2))))
    print("R2 model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_r2'].mean(), 2))))
    print("-------------------------")
    print("MSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr), 2))))
    print("RMSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))))
    print("MAE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_lr), 2))))
    print("R2 model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_lr), 2))))
    print(" ")
    
    if(abs(round(mean_squared_error(y_test, y_pred_lr), 2) < min_mse)):
        min_mse = abs(round(mean_squared_error(y_test, y_pred_lr), 2))

    if(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2) < min_rmse)):
        min_rmse = abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))

    if(abs(round(mean_absolute_error(y_test, y_pred_lr), 2) < min_mae)):
        min_mae = abs(round(mean_absolute_error(y_test, y_pred_lr), 2))

    end_time = time.time()
    print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Linear Regression data Train dengan 1 fitur: 188.01
RMSE model Linear Regression data Train dengan 1 fitur:13.71
MAE model Linear Regression data Train dengan 1 fitur:11.06
R2 model Linear Regression data Train dengan 1 fitur:0.02
-------------------------
MSE model Linear Regression data Test dengan 1 fitur:184.09
RMSE model Linear Regression data Test dengan 1 fitur:13.57
MAE model Linear Regression data Test dengan 1 fitur:10.44
R2 model Linear Regression data Test dengan 1 fitur:0.07
 
Total waktu:  0.20164728164672852
MSE model Linear Regression data Train dengan 2 fitur: 174.55
RMSE model Linear Regression data Train dengan 2 fitur:13.21
MAE model Linear Regression data Train dengan 2 fitur:10.64
R2 model Linear Regression data Train dengan 2 fitur:0.09
-------------------------
MSE model Linear Regression data Test dengan 2 fitur:197.18
RMSE model Linear Regression data Test dengan 2 fitur:14.04
MAE model Linear Regression data Test dengan 2 fitur:10.86
R2 model Linear

In [10]:
scores

{'fit_time': array([0.04786277, 0.00727606, 0.00787163, 0.04899502, 0.00797749,
        0.00892758, 0.04775023, 0.00794983, 0.00793624, 0.04834723]),
 'score_time': array([0.0065887 , 0.00479412, 0.00478172, 0.00779104, 0.00478435,
        0.00477982, 0.00733304, 0.0047667 , 0.00476789, 0.00732565]),
 'test_rmse': array([-2.88186157e+08, -5.34880460e+07, -1.96889509e+07, -4.01943003e+08,
        -2.20354908e+08, -3.24237131e+08, -2.91168000e+08, -5.70096131e+07,
        -1.39483587e+09, -2.00878437e+08]),
 'train_rmse': array([-4.69535019, -4.29324469, -5.16197377, -5.14858044, -5.09566549,
        -4.34046483, -5.08613331, -4.67617207, -4.38936339, -5.29390108]),
 'test_mse': array([-8.30512613e+16, -2.86097107e+15, -3.87654788e+14, -1.61558178e+17,
        -4.85562855e+16, -1.05129717e+17, -8.47788044e+16, -3.25009598e+15,
        -1.94556712e+18, -4.03521465e+16]),
 'train_mse': array([-22.04631338, -18.43194999, -26.64597319, -26.50788052,
        -25.96580674, -18.83963491, -25.86

# Random Forest Regression (n = 100)

In [11]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

#tentukan total fitur yang digunakan dalam proses klasifikasi SVM ini
n_feat = range(1, 101)
n_trees = [100]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        #ambil n fitur input hasil seleksi fitur Anova
        x_train_selected = x_train[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]
        x_test_selected = x_test[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:49.99
RMSE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:7.06
MAE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:5.52
R2 model Random Forest Regression data Train dengan 1 fitur dan 100 trees:0.74
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:236.96
RMSE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:15.39
MAE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:12.49
R2 model Random Forest Regression data Test dengan 1 fitur dan 100 trees:0.37
 
Total waktu:  1.2138690948486328
MSE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:49.08
RMSE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:7.0
MAE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:5.51
R2 model Random Forest Regression data Train dengan 2 fit

# Random Forest Regression n = 150

In [12]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [150]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        #ambil n fitur input hasil seleksi fitur Anova
        x_train_selected = x_train[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]
        x_test_selected = x_test[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:50.42
RMSE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:7.1
MAE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:5.55
R2 model Random Forest Regression data Train dengan 1 fitur dan 150 trees:0.74
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:241.23
RMSE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:15.53
MAE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:12.65
R2 model Random Forest Regression data Test dengan 1 fitur dan 150 trees:0.4
 
Total waktu:  1.6904118061065674
MSE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:48.58
RMSE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:6.97
MAE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:5.49
R2 model Random Forest Regression data Train dengan 2 fitu

# Random Forest Regression n = 200

In [13]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [200]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        #ambil n fitur input hasil seleksi fitur Anova
        x_train_selected = x_train[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]
        x_test_selected = x_test[features_anova.nlargest(100,columns="Score").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:49.57
RMSE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:7.04
MAE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:5.52
R2 model Random Forest Regression data Train dengan 1 fitur dan 200 trees:0.74
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:250.36
RMSE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:15.82
MAE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:12.83
R2 model Random Forest Regression data Test dengan 1 fitur dan 200 trees:0.45
 
Total waktu:  2.2796759605407715
MSE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:49.08
RMSE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:7.0
MAE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:5.47
R2 model Random Forest Regression data Train dengan 2 fit